## 1. Introduction

Customer churn is a critical challenge for telecommunications companies, as losing existing customers directly impacts revenue and increases acquisition costs. Understanding which customers are likely to churn, and the reasons behind their decisions, enables organizations to design targeted and cost-effective retention strategies.

This project analyzes a telecom customer dataset containing demographic attributes, service subscriptions, billing and revenue information, and churn outcomes. Each row represents an individual customer, with the churn label indicating whether the customer left the company.

The objective of this project is to build an end-to-end analytics and machine learning solution that:
- Identifies customers at high risk of churn,
- Understands the key drivers and patterns associated with churn behavior, and
- Translates analytical insights into actionable business recommendations to improve customer retention.


## 2. Dataset Overview

The dataset represents a realistic telecom customer environment and includes the following feature groups:

Data Set From Kaggel :

https://www.kaggle.com/datasets/alfathterry/telco-customer-churn-11-1-3

- **Customer Demographics:** Age, gender, marital status, dependents, geography
- **Service Subscriptions:** Phone service, internet service, streaming options, contract type
- **Usage & Billing:** Monthly charges, total charges, long-distance usage
- **Revenue Metrics:** Total revenue, refunds, extra data charges, customer lifetime value (CLTV)
- **Churn Information:** Churn label, churn score, churn category, and churn reason

The primary target variable for modeling is **Churn Label**, where:
- `Yes` indicates a churned customer
- `No` indicates a retained customer


### Import Libraries (Code)

In [2]:
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

### Load the Dataset (Code)

In [3]:
df = pd.read_csv('../data/raw/telco_churn_dataset.csv')
df.head()

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Country,State,City,Zip Code,Latitude,Longitude,Population,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Label,Churn Score,CLTV,Churn Category,Churn Reason
0,8779-QRDMV,Male,78,No,Yes,No,No,0,United States,California,Los Angeles,90022,34.023810,-118.156582,68701,Q3,No,0,1,NaN,No,0.00,No,Yes,DSL,8,No,No,Yes,No,No,Yes,No,No,Month-to-Month,Yes,Bank Withdrawal,39.65,39.65,0.00,20,0.00,59.65,3,Churned,Yes,91,5433,Competitor,Competitor offered more data
1,7495-OOKFY,Female,74,No,Yes,Yes,Yes,1,United States,California,Los Angeles,90063,34.044271,-118.185237,55668,Q3,Yes,1,8,Offer E,Yes,48.85,Yes,Yes,Fiber Optic,17,No,Yes,No,No,No,No,No,Yes,Month-to-Month,Yes,Credit Card,80.65,633.30,0.00,0,390.80,1024.10,3,Churned,Yes,69,5302,Competitor,Competitor made better offer
2,1658-BYGOY,Male,71,No,Yes,No,Yes,3,United States,California,Los Angeles,90065,34.108833,-118.229715,47534,Q3,No,0,18,Offer D,Yes,11.33,Yes,Yes,Fiber Optic,52,No,No,No,No,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,1752.55,45.61,0,203.94,1910.88,2,Churned,Yes,81,3179,Competitor,Competitor made better offer
3,4598-XLKNJ,Female,78,No,Yes,Yes,Yes,1,United States,California,Inglewood,90303,33.936291,-118.332639,27778,Q3,Yes,1,25,Offer C,Yes,19.76,No,Yes,Fiber Optic,12,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.50,2514.50,13.43,0,494.00,2995.07,2,Churned,Yes,88,5337,Dissatisfaction,Limited range of services
4,4846-WHAFZ,Female,80,No,Yes,Yes,Yes,1,United States,California,Whittier,90602,33.972119,-118.020188,26265,Q3,Yes,1,37,Offer C,Yes,6.33,Yes,Yes,Fiber Optic,14,No,No,No,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,76.50,2868.15,0.00,0,234.21,3102.36,2,Churned,Yes,67,2793,Price,Extra data charges


## Column Name Standardization

To improve code readability and consistency across analysis and modeling steps, all column names were standardized to lowercase and converted to snake_case format. This practice aligns with common data engineering and analytics conventions and helps avoid potential issues related to spacing or case sensitivity.


In [4]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
      .str.replace("-", "_")
)


## Initial Data Inspection

In [5]:
df.shape

(7043, 50)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 50 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   customer_id                        7043 non-null   object 
 1   gender                             7043 non-null   object 
 2   age                                7043 non-null   int64  
 3   under_30                           7043 non-null   object 
 4   senior_citizen                     7043 non-null   object 
 5   married                            7043 non-null   object 
 6   dependents                         7043 non-null   object 
 7   number_of_dependents               7043 non-null   int64  
 8   country                            7043 non-null   object 
 9   state                              7043 non-null   object 
 10  city                               7043 non-null   object 
 11  zip_code                           7043 non-null   int64

In [7]:
df.describe(include='all')

,customer_id,gender,age,under_30,senior_citizen,married,dependents,number_of_dependents,country,state,city,zip_code,latitude,longitude,population,quarter,referred_a_friend,number_of_referrals,tenure_in_months,offer,phone_service,avg_monthly_long_distance_charges,multiple_lines,internet_service,internet_type,avg_monthly_gb_download,online_security,online_backup,device_protection_plan,premium_tech_support,streaming_tv,streaming_movies,streaming_music,unlimited_data,contract,paperless_billing,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,satisfaction_score,customer_status,churn_label,churn_score,cltv,churn_category,churn_reason
count,7043,7043,7043.000000,7043,7043,7043,7043,7043.000000,7043,7043,7043,7043.000000,7043.000000,7043.000000,7043.000000,7043,7043,7043.000000,7043.000000,3166,7043,7043.000000,7043,7043,5517,7043.000000,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043,7043,7043.000000,7043.000000,1869,1869
unique,7043,2,NaN,2,2,2,2,NaN,1,1,1106,NaN,NaN,NaN,NaN,1,2,NaN,NaN,5,2,NaN,2,2,3,NaN,2,2,2,2,2,2,2,2,3,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,2,NaN,NaN,5,20
top,8779-QRDMV,Male,NaN,No,No,No,No,NaN,United States,California,Los Angeles,NaN,NaN,NaN,NaN,Q3,No,NaN,NaN,Offer B,Yes,NaN,No,Yes,Fiber Optic,NaN,No,No,No,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stayed,No,NaN,NaN,Competitor,Competitor had better devices
freq,1,3555,NaN,5642,5901,3641,5416,NaN,7043,7043,293,NaN,NaN,NaN,NaN,7043,3821,NaN,NaN,824,6361,NaN,4072,5517,3035,NaN,5024,4614,4621,4999,4336,4311,4555,4745,3610,4171,3909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4720,5174,NaN,NaN,841,313
mean,NaN,NaN,46.509726,NaN,NaN,NaN,NaN,0.468692,NaN,NaN,NaN,93486.070567,36.197455,-119.756684,22139.603294,NaN,NaN,1.951867,32.386767,NaN,NaN,22.958954,NaN,NaN,NaN,20.515405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,2280.381264,1.962182,6.860713,749.099262,3034.379056,3.244924,NaN,NaN,58.505040,4400.295755,NaN,NaN
std,NaN,NaN,16.750352,NaN,NaN,NaN,NaN,0.962802,NaN,NaN,NaN,1856.767505,2.468929,2.154425,21152.392837,NaN,NaN,3.001199,24.542061,NaN,NaN,15.448113,NaN,NaN,NaN,20.418940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,2266.220462,7.902614,25.104978,846.660055,2865.204542,1.201657,NaN,NaN,21.170031,1183.057152,NaN,NaN
min,NaN,NaN,19.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,90001.000000,32.555828,-124.301372,11.000000,NaN,NaN,0.000000,1.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,18.800000,0.000000,0.000000,0.000000,21.360000,1.000000,NaN,NaN,5.000000,2003.000000,NaN,NaN
25%,NaN,NaN,32.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,92101.000000,33.990646,-121.788090,2344.000000,NaN,NaN,0.000000,9.000000,NaN,NaN,9.210000,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,400.150000,0.000000,0.000000,70.545000,605.610000,3.000000,NaN,NaN,40.000000,3469.000000,NaN,NaN
50%,NaN,NaN,46.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,93518.000000,36.205465,-119.595293,17554.000000,NaN,NaN,0.000000,29.000000,NaN,NaN,22.890000,NaN,NaN,NaN,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,1394.550000,0.000000,0.000000,401.440000,2108.640000,3.000000,NaN,NaN,61.000000,4527.000000,NaN,NaN
75%,NaN,NaN,60.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,95329.000000,38.161321,-117.969795,36125.000000,NaN,NaN,3.000000,55.000000,NaN,NaN,36.395000,NaN,NaN,NaN,27.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,3786.600000,0.000000,0.000000,1191.100000,4801.145000,4.000000,NaN,NaN,75.500000,5380.500000,NaN,NaN


#### Discovered

- The dataset has 7043 rows and  50 columns
- There no numerical missing values but contextual in columns as ('internet type , churn category and churn reason)
- Which columns are categorical vs numerical? - i don't know
- Which variables appear derived or post-event? - i don't know


## Target Variable Validation
- Making sure churn predicting is correct, meaningful, and usable before building models

In [8]:
df['churn_label'].value_counts()

churn_label
No     5174
Yes    1869
Name: count, dtype: int64

In [9]:
df['churn_label'].value_counts(normalize=True)

churn_label
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64

The churn distribution indicates that approximately 25–30% of customers in the dataset have churned. This reflects a moderately imbalanced target variable, which is common in telecom churn problems.

## Target Variable Encoding

For consistency in analysis and to support statistical aggregation and predictive modeling, the churn target variable was encoded as a binary label. Customers who churned were mapped to 1, while retained customers were mapped to 0. This encoding enables intuitive interpretation of group-level means as churn rates and ensures compatibility with machine learning algorithms.


In [10]:
df['churn_label']= df['churn_label'].map({'No':0,'Yes':1})

## Feature Categorization & Data Leakage Prevention

To ensure that predictive modeling reflects real-world deployment scenarios, post-outcome variables and derived churn indicators were excluded from modeling to prevent data leakage.
These variables contain information that would not be available at the time of prediction and could artificially inflate model performance.
Such variables were retained solely for exploratory analysis and business interpretation.


In [11]:
# excluding post-churn and outcome-derived variables
leakage_columns = [
    "customer_status",
    "churn_score",
    "churn_category",
    "churn_reason"
]

identifier_columns = ["customer_id"]

df[leakage_columns].head()

,customer_status,churn_score,churn_category,churn_reason
0,Churned,91,Competitor,Competitor offered more data
1,Churned,69,Competitor,Competitor made better offer
2,Churned,81,Competitor,Competitor made better offer
3,Churned,88,Dissatisfaction,Limited range of services
4,Churned,67,Price,Extra data charges


## Sanity Checks & Business Validation

In [12]:
df[["tenure_in_months", "monthly_charge", "total_charges", "total_revenue"]].describe()


,tenure_in_months,monthly_charge,total_charges,total_revenue
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,32.386767,64.761692,2280.381264,3034.379056
std,24.542061,30.090047,2266.220462,2865.204542
min,1.000000,18.250000,18.800000,21.360000
25%,9.000000,35.500000,400.150000,605.610000
50%,29.000000,70.350000,1394.550000,2108.640000
75%,55.000000,89.850000,3786.600000,4801.145000
max,72.000000,118.750000,8684.800000,11979.340000


In [13]:
(df["total_charges"] < 0).sum()


np.int64(0)

In [14]:
(df["monthly_charge"] <= 0).sum()


np.int64(0)

In [15]:
(df["total_revenue"] < df["monthly_charge"]).sum()


np.int64(0)

Basic sanity checks were performed on tenure, billing, and revenue variables to ensure alignment with real-world business logic. No negative values were observed for monetary fields, and tenure values fell within reasonable ranges. These checks confirm that the dataset is internally consistent and suitable for further analysis.


## Data Cleaning Decisions

1. The dataset required minimal preprocessing as it was already well-structured and contained no significant missing values.
2. Cleaning efforts focused on standardizing column names, validating data consistency, and ensuring alignment with real-world business logic.
3. Post-outcome variables related to churn were retained for exploratory analysis and interpretation but deliberately excluded from predictive modeling to prevent data leakage.
4. No records were removed at this stage to preserve the integrity of the original customer population.


## Save Cleaned Dataset

In [17]:
df_clean= df.copy()

df_clean.to_csv("../data/processed/telco_churn_cleaned.csv", index=False)

## Cleaning Summary

This phase focused on establishing a reliable foundation for analysis by understanding the dataset structure, validating the churn target, and ensuring data quality and business coherence.
- Column names were standardized for consistency,
- key feature groups were identified,
 - potential sources of data leakage were explicitly documented and excluded from modeling considerations.
- Sanity checks confirmed that tenure, billing, and revenue variables aligned with realistic telecom business logic.
- The resulting cleaned and well-defined dataset provides a solid and trustworthy basis for exploratory analysis and predictive modeling in the next stages.
